# Experiment on VGG
Fine tune VGG16 on Fashion MNIST cause i don't have access to DeepFashion2 yet

<img src="../assets/image.png" alt="image" width="200"/>

## Load pretrained VGG16

In [1]:
import torch
import torchvision.models as models

# Load pretrained VGG16 model
vgg16 = models.vgg11(pretrained=True)

# To use without pretrained weights:
# vgg16 = models.vgg16(pretrained=False)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

# Example: modify classifier for custom 10-class task
vgg16.classifier[6] = torch.nn.Linear(4096, 10)

c:\Users\cpick\Documents\GitHub\SlowFashionProject\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\cpick\Documents\GitHub\SlowFashionProject\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to C:\Users\cpick/.cache\torch\hub\checkpoints\vgg11-8a719046.pth


100%|██████████| 507M/507M [03:14<00:00, 2.73MB/s] 


## Load dataset and augment data

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. Define the augmentation pipeline
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),      # Random flip
    transforms.RandomRotation(10),           # Rotate images by +/- 10 degrees
    transforms.RandomAffine(0, translate=(0.1, 0.1)),  # Translate up to 10% height/width
    transforms.Resize(224),       # Resize to VGG expected size
    transforms.Grayscale(num_output_channels=3),  # Convert 1-channel to 3
    transforms.ToTensor(),                   # Convert PIL image to tensor
    transforms.Normalize((0.5,), (0.5,))      # Normalize pixel values to [-1, 1]
])

test_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
])

# 2. Load datasets with transform
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.0001) # Only classifier is trainable

: 

## Train

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    vgg16.train()
    total_loss = 0
    correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")


## Eval

In [ ]:
vgg16.eval()
correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16(images)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / len(test_dataset)
print(f"Test Accuracy: {test_acc:.2f}%")